|        |        |        |
|--------|--------|--------|
![H-BRS](logos/h-brs.png) | ![A2S](logos/a2s.png) | ![b-it](logos/b-it.png) |

# Autonomous Mobile Robots

# AMR Assignment 5

### General information

* Please do not add or delete any cells. Answers belong into the already provided cells (below the question).
* If a function is given (either as a signature or a full function), you should not change the name, arguments, or return value of the function.
* If you encounter empty cells underneath the answer that can not be edited, please ignore them; they are for testing purposes.
* Please note that variables declared in the notebook cells have global scope. To make sure your assignment works correctly as a whole, please restart the kernel and run all cells before submitting (e.g. via *Kernel -> Restart & Run All*).
* Code cells where you are supposed to give your answer often include the line  ```raise NotImplementedError```. This makes it easier to automatically grade answers. Once you fill out a function, please delete this line.

### Submission

Please make sure to write all your team members 2s IDs in the cell below before submission. Please submit your notebook via the JupyterHub web interface (in the main view -> Assignments -> Submit). If it is a group assignment, please make only one submission per group (for easier bookkeeping, it is best if this is always the same team member).

### Questions about the assignment

If you have questions about the assignment, you are encouraged to post them in the LEA forum. Proactive discussions lead to better understanding. Let's keep the forum active.

## Team members (2s IDs):

YOUR ANSWER HERE

*anuhel2s*

*mkhan2s*

*spatha2s*

# Landmark-based Localisation using Kalman Filter (100 points)

In this assignment, we will use a Kalman filter for localisation in an environment based on the measurement of landmark locations. As you already know, sensor readings are affected by noise, making it difficult to achieve accurate measurement of the physical quantity under consideration. The Kalman filter, which belongs to the family of Bayes filter algorithms, can be used for minimising the error in the estimation. The belief of the state (in this case, it is the **pose of the robot** with respect to the **map frame**) is represented as a Gaussian distribution. The objective is to estimate the mean $\mu$ and covariance matrix $\Sigma$ for the physical quantity under consideration at every time instance.

In this assignment, the Robile is placed in a simulation environment in Gazebo, where several RFID tags are positioned on the ground. Now, suppose that a virtual RFID detector is mounted on the robot, sharing the *same frame* and *field of view* as the LiDAR. In the real world, the robot's real pose almost never matches with its ideal motion model due to different sources of noise, such as from the encoder readings, unstructured environment interaction, as well as other noisy perception data.

In this assignment, we will explore how the Kalman filter can be employed to improve the estimation of the robot's pose in this setup. Below is the simulation environment where the RFID tags are highlighted.

![robile_localisation_kalman](img/robile_localisation_kalman.png)

The positions of the RFID tags as $(x,y)$ coordinates in the `odom` frame with tag names are as follows:  
- A: $(1.0, 1.0)$
- B: $(6.0, 1.0)$
- C: $(3.0, -1.0)$
- D: $(1.0, -3.0)$
- E: $(4.0, -4.0)$

### Motion noise consideration

When the robot is moving in the environment, the noise in the data published onto the `/odom` topic is negligible in the simulation. To simulate the real-world noise, for every message on the `/odom` topic, some noise is introduced manually in the associated *robile_rfid_tag_finder.py* script, and the variable `real_base_link_pose` is updated accordingly. Here, the data on the `/odom` topic, and consequently the visualization of the robot model in RVIZ, can be considered as an ideal motion model, which is the motion estimate without noise; the data on `real_base_link_pose` is then the actual pose of the robot's base (`real_base_link` frame) with respect to which the RFID tags are measured, and which are available on the `/rfid_tag_poses` topic. The `real_base_link_pose` topic thus represents the actual pose of the robot, which deviates from `/odom` due to uncertain interaction with the environment.

Your goal is to read from the `odom` topic and, along with the measured RFID tag positions, estimate the posterior belief of the robot's pose and the variance associated with it. This estimated pose, which should be published to the `estimated_base_link_pose` topic, should eventually converge to the pose represented on `real_laser_link_pose`.

### Summary

- The pose of the robot with respect to the odom frame, published on the `/odom` topic represents ideal motion without noise.
- Since it is challenging to mathematically represent or model all physical interactions with the environment, in reality, the robot deviates from the ideal motion and its pose estimation is affected by noise in the sensor data and an incomplete model of the interaction with the environment. The actual robot pose is considered to be the `real_base_link` frame.
- The measurement model is the pose estimate using the detected RFID tags on the `/rfid_tag_poses` topic, which is represented by a custom message of type ``PoseLabelledArray``. These tags are measured with respect to the `real_base_link` frame.
- Note that the `real_base_link` frame is only used to visualise the actual robot pose; thus, only the data on `/odom` and `/rfid_tag_poses` topics should be used to determine robot's posterior pose `estimated_base_link_pose`.


### Notes

- The initial `real_base_link` frame is considered to be overlapping with the `odom` frame.
- To determine the robot's pose by the measurement of RFID tag positions, at least two tags needs to be perceived. The problem could be made simpler by considering the pose estimated using these RFID positions as measurement data rather than the measured RFID positions themselves.
- The initial state, i.e., the `estimated_base_link_pose` pose at the start of simulation is considered the same as the pose of the `odom` frame, namely `estimated_base_link_pose` can be considered as initially known with zero or constant uncertainty. Optionally, it can be updated using the `2D Pose Estimate` widget in RVIZ, in which case the mean and variance of the estimate should be reset.

### Instructions

Upload the modified `robile_navigation` package with your submission, and also paste your Kalman filter implementation in the cell below. Before starting your implementation please follow these steps:

- Modify the simulation launch file to load the `closed_walls.world` environment from the `robile_gazebo` package
- Implement pose estimation using a Kalman filter in the `robile_localisation_kalman.py` script from the `robile_navigation` repository
- Use the launch file `landmark_based_localisation.launch.py` under `robile_navigation` and modify it accordingly to launch your implementation
- Please feel free to modify any of the scripts according to your needs

### Potentially useful references

- Section 5.6.8.5 in the "Introduction to Autonomous Mobile Robots" book. It doesn't have the same setup, but is still useful
- https://www.alanzucconi.com/2022/07/24/kalman-gain/
- http://bilgin.esme.org/BitsAndBytes/KalmanFilterforDummies

In [ ]:
### Paste your code from robile_localisation_kalman.py in this cell

# YOUR CODE HERE
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import PoseStamped
from robile_interfaces.msg import PositionLabelledArray
from nav_msgs.msg import Odometry
import numpy as np
from tf_transformations import euler_from_quaternion, quaternion_from_euler

class LocalisationUsingKalmanFilter(Node):
    def __init__(self):
        super().__init__('localisation_using_kalman_filter')

        # Declare and fetch parameters
        self.declare_parameters(namespace='', parameters=[
            ('map_frame', 'map'),
            ('odom_frame', 'odom'),
            ('rfid_tag_poses_topic', 'rfid_tag_poses'),
            ('odom_topic', 'odom'),
            ('estimated_base_link_pose_topic', 'estimated_base_link_pose'),
            ('rfid_tags.A', [1.0, 1.0]),
            ('rfid_tags.B', [6.0, 1.0]),
            ('rfid_tags.C', [3.0, -1.0]),
            ('rfid_tags.D', [1.0, -3.0]),
            ('rfid_tags.E', [4.0, -4.0]),
        ])

        self.map_frame = self.get_parameter('map_frame').get_parameter_value().string_value
        self.rfid_tag_poses_topic = self.get_parameter('rfid_tag_poses_topic').get_parameter_value().string_value
        self.odom_topic = self.get_parameter('odom_topic').get_parameter_value().string_value
        self.estimated_base_link_pose_topic = self.get_parameter('estimated_base_link_pose_topic').get_parameter_value().string_value
        self.rfid_tags = {tag: self.get_parameter(f'rfid_tags.{tag}').get_parameter_value().double_array_value
                          for tag in ['A', 'B', 'C', 'D', 'E']}

        # Subscribers and publishers
        self.create_subscription(PositionLabelledArray, self.rfid_tag_poses_topic, self.rfid_callback, 10)
        self.create_subscription(Odometry, self.odom_topic, self.odom_callback, 10)
        self.estimated_pose_publisher = self.create_publisher(PoseStamped, self.estimated_base_link_pose_topic, 10)

        # Kalman filter variables
        self.x = np.zeros(3)  # [x, y, theta]
        self.P = np.eye(3) * 0.1  # Initial covariance
        self.Q = np.eye(3) * 0.1  # Process noise covariance
        self.R = np.eye(4) * 0.5  # Measurement noise covariance
        self.last_odom_pose = None
        self.measurements = []  # To store RFID measurements
        self.landmarks = []     # To store corresponding landmarks

    def normalize_angle(self, angle):
        return (angle + np.pi) % (2 * np.pi) - np.pi

    def odom_callback(self, msg):
        position = msg.pose.pose.position
        orientation = msg.pose.pose.orientation
        _, _, theta = euler_from_quaternion([orientation.x, orientation.y, orientation.z, orientation.w])

        if self.last_odom_pose is None:
            self.last_odom_pose = (position.x, position.y, theta)
            return

        dx = position.x - self.last_odom_pose[0]
        dy = position.y - self.last_odom_pose[1]
        dtheta = self.normalize_angle(theta - self.last_odom_pose[2])

        self.last_odom_pose = (position.x, position.y, theta)
        self.predict(dx, dy, dtheta)

        # Synchronize state update with measurement update if measurements are available
        if self.measurements and len(self.measurements) >= 2:
            self.update(self.measurements, self.landmarks)
            self.measurements.clear()
            self.landmarks.clear()

    def predict(self, dx, dy, dtheta):
        """
        Kalman Filter prediction step.
        """
        theta = self.x[2]
        
        # State transition matrix (Identity matrix for this scenario)
        A = np.eye(3)

        # Control input matrix
        B = np.array([
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1]
        ])

        u = np.array([dx, dy, dtheta])
        self.x = A @ self.x + B @ u
        self.P = A @ self.P @ A.T + self.Q

    def rfid_callback(self, msg):
        for data in msg.positions:
            if data.name in self.rfid_tags:
                self.measurements.append([data.position.x, data.position.y])
                self.landmarks.append(self.rfid_tags[data.name])

    def update(self, measurements, landmarks):
        """
        Kalman Filter measurement update step.
        """
        Z = np.array(measurements).flatten()
        H = self.calculate_jacobian(landmarks)

        # Innovation covariance
        S = H @ self.P @ H.T + self.R
        K = self.P @ H.T @ np.linalg.inv(S)  # Kalman gain

        # Innovation (measurement residual)
        z_hat = H @ self.x  # Expected measurement
        y = Z - z_hat       # Residual

        # State update
        self.x = self.x + K @ y

        # Covariance update
        I = np.eye(len(self.x))
        self.P = (I - K @ H) @ self.P

        self.publish_estimated_pose()

    def calculate_jacobian(self, landmarks):
        """
        Calculate the Jacobian matrix for the measurement model.
        """
        H = []
        for lx, ly in landmarks:
            px, py, theta = self.x
            H.append([1, 0, 0])  # Mapping x
            H.append([0, 1, 0])  # Mapping y
        return np.array(H).reshape(len(landmarks) * 2, 3)

    def publish_estimated_pose(self):
        """
        Publish the estimated pose to the estimated_base_link_pose topic.
        """
        pose = PoseStamped()
        pose.header.stamp = self.get_clock().now().to_msg()
        pose.header.frame_id = self.map_frame
        pose.pose.position.x = self.x[0]
        pose.pose.position.y = self.x[1]
        q = quaternion_from_euler(0, 0, self.x[2])
        pose.pose.orientation.x, pose.pose.orientation.y, pose.pose.orientation.z, pose.pose.orientation.w = q
        self.estimated_pose_publisher.publish(pose)


def main(args=None):
    rclpy.init(args=args)
    node = LocalisationUsingKalmanFilter()
    try:
        rclpy.spin(node)
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()


Finally, comment on your observations while evaluating your implementation in the cell below, and include relevant screenshots to verify your implementation.

During the evaluation of my implementation, the Kalman filter-based localization worked as expected, with the robot successfully estimating its pose using RFID tag detections. The transforms were properly established, and the robot's motion was accurately tracked in the Gazebo simulation.